# **Load and Clean**

In [34]:
# Load All

import os
import glob
import pandas as pd

# Path to your data
path = 'GoogleReviews-20240420T223639Z-001/GoogleReviews' 
# Get all .csv files at your path
all_files = glob.glob(os.path.join(path, "*.csv"))

# Initialize a list to store dataframes
dataframes = []

# Define a function to standardize column names and handle special cases
def standardize_columns(df, filename):
    # Rename columns to create a consistent schema
    column_renames = {
        'author_id': 'user_id',
        'author_title': 'author',
        'review_datetime_utc': 'timestamp',
    }
    df.rename(columns={old: new for old, new in column_renames.items() if old in df.columns}, inplace=True)

    # Check if 'rating' column exists; if it does, this is a special case file
    if 'rating' in df.columns and 'review_text' in df.columns:
        # Switch 'rating' and 'review_text'
        df['rating'], df['review_text'] = df['review_text'], df['rating']

    # Combine 'rating' and 'review_rating' into one column, preferring 'rating' if it exists
    if 'rating' in df.columns and 'review_rating' in df.columns:
        df['rating'] = df['rating'].fillna(df['review_rating'])
        df.drop(columns=['review_rating'], inplace=True, errors='ignore')
    elif 'review_rating' in df.columns:
        df.rename(columns={'review_rating': 'rating'}, inplace=True)
        
    # Add missing columns with NaNs
    desired_columns = ['user_id', 'author', 'timestamp', 'rating', 'review_text']
    for col in desired_columns:
        if col not in df.columns:
            df[col] = pd.NA

    # Add the filename as a new column
    df['file_name'] = filename.split('/')[-1]  # Get just the file name

    return df

# Process each file
for filename in all_files:
    df = pd.read_csv(filename)
    
    # Standardize column names and handle special cases
    df = standardize_columns(df, filename)
    
    # Append to the list of dataframes
    dataframes.append(df)

# Concatenate all dataframes
reviews_df = pd.concat(dataframes, ignore_index=True)

In [35]:
reviews_df.head()

,timestamp,author,user_id,review_text,rating,file_name
0,02/11/2024 15:59:38,Heidy Fernandez,103360372037270387417,NaN,5,GoogleReviews\ChIJ-5kFm5vIxokR8VHXewCN8eY_Dela...
1,01/26/2024 22:16:50,R J,103202524746831207554,NaN,5,GoogleReviews\ChIJ-5kFm5vIxokR8VHXewCN8eY_Dela...
2,12/30/2023 19:08:54,Eleni Belisonzi,106030427780813104531,A neighbor jewel.,5,GoogleReviews\ChIJ-5kFm5vIxokR8VHXewCN8eY_Dela...
3,09/05/2023 14:40:24,Tom Donald,110477736284587831432,NaN,5,GoogleReviews\ChIJ-5kFm5vIxokR8VHXewCN8eY_Dela...
4,08/18/2023 19:13:19,Evan Gray,117127716926195155753,NaN,5,GoogleReviews\ChIJ-5kFm5vIxokR8VHXewCN8eY_Dela...


In [36]:
def extract_place_id(file_name):
    return file_name.split('\\')[-1][:27]

reviews_df['place_id'] = reviews_df['file_name'].apply(extract_place_id)

In [37]:
reviews_df.head()

,timestamp,author,user_id,review_text,rating,file_name,place_id
0,02/11/2024 15:59:38,Heidy Fernandez,103360372037270387417,NaN,5,GoogleReviews\ChIJ-5kFm5vIxokR8VHXewCN8eY_Dela...,ChIJ-5kFm5vIxokR8VHXewCN8eY
1,01/26/2024 22:16:50,R J,103202524746831207554,NaN,5,GoogleReviews\ChIJ-5kFm5vIxokR8VHXewCN8eY_Dela...,ChIJ-5kFm5vIxokR8VHXewCN8eY
2,12/30/2023 19:08:54,Eleni Belisonzi,106030427780813104531,A neighbor jewel.,5,GoogleReviews\ChIJ-5kFm5vIxokR8VHXewCN8eY_Dela...,ChIJ-5kFm5vIxokR8VHXewCN8eY
3,09/05/2023 14:40:24,Tom Donald,110477736284587831432,NaN,5,GoogleReviews\ChIJ-5kFm5vIxokR8VHXewCN8eY_Dela...,ChIJ-5kFm5vIxokR8VHXewCN8eY
4,08/18/2023 19:13:19,Evan Gray,117127716926195155753,NaN,5,GoogleReviews\ChIJ-5kFm5vIxokR8VHXewCN8eY_Dela...,ChIJ-5kFm5vIxokR8VHXewCN8eY


In [38]:
reviews_df.to_csv('combined_reviews.csv', index=False)

In [39]:
ppr_df = pd.read_csv('PPR-parks-April17.csv')

In [40]:
reviews_df = reviews_df.merge(ppr_df[['GoogleID', 'ParkName', 'Lat', 'Long']], 
                                left_on='place_id', 
                                right_on='GoogleID', 
                                how='left')

# Drop the now redundant 'GoogleID' column
reviews_df.drop('GoogleID', axis=1, inplace=True)

# Save the merged dataframe to a new CSV file, if needed
reviews_df.to_csv('combined_reviews_with_park_info.csv', index=False)

In [41]:
FDR = "Tacony Creek Park"

# Create a list of place_ids to update based on the second image
FDR_place_ids_to_update = [
    'ChIJjQO6UDTFxokRKwipjcz5Q5o',
    'ChIJB37Luk_FxokRSxEC_QgGmAU',
    'ChIJm1IODzjFxokREo4XLuKy_WQ',
    'ChIJa8NFISXFxokRyfcKW9fOUTU',
    'ChIJQ0ZFt63FxokRqkfAi0r4wPQ',
    'ChIJobR0mq63xokRiBpks2cGAo0',
    'ChIJZTP18zXFxokRZSENwg8ci8o']

for pid in FDR_place_ids_to_update:
    reviews_df.loc[reviews_df['place_id'] == pid, 'ParkName'] = FDR

In [42]:
len(reviews_df[reviews_df['ParkName'] == 'Franklin Delano Roosevelt Park'])

## orginal 4338, should worked

4352

In [43]:
Tacony = "Tacony Creek Park"

# Create a list of place_ids to update based on the second image
Tacony_place_ids_to_update = [
    'ChIJcf8po0u2xokRjKskdO6gjg4',
    'ChIJ26qgUku2xokRWu2KHZAoxso',
    'ChIJd_QXFbK3xokR6p-tgQG1jsA',
    'ChIJp-fcuq63xokReHv464Y-4jw',
    'ChIJpRYFA0u2xokRqCSGUUT3NTc',
    'ChIJobR0mq63xokRiBpks2cGAo0',
    'ChIJ_UE-k022xokRkn8Tn8lEZo0']

for pid in Tacony_place_ids_to_update:
    reviews_df.loc[reviews_df['place_id'] == pid, 'ParkName'] = Tacony

In [51]:
len(reviews_df[reviews_df['ParkName'] == 'Tacony Creek Park'])

## originally 1759

2260

In [46]:
len(reviews_df)

86007

In [48]:
reviews_df.head()

,timestamp,author,user_id,review_text,rating,file_name,place_id,ParkName,Lat,Long
0,02/11/2024 15:59:38,Heidy Fernandez,103360372037270387417,NaN,5,GoogleReviews\ChIJ-5kFm5vIxokR8VHXewCN8eY_Dela...,ChIJ-5kFm5vIxokR8VHXewCN8eY,Delancey Park,39.94408,-75.1479
1,01/26/2024 22:16:50,R J,103202524746831207554,NaN,5,GoogleReviews\ChIJ-5kFm5vIxokR8VHXewCN8eY_Dela...,ChIJ-5kFm5vIxokR8VHXewCN8eY,Delancey Park,39.94408,-75.1479
2,12/30/2023 19:08:54,Eleni Belisonzi,106030427780813104531,A neighbor jewel.,5,GoogleReviews\ChIJ-5kFm5vIxokR8VHXewCN8eY_Dela...,ChIJ-5kFm5vIxokR8VHXewCN8eY,Delancey Park,39.94408,-75.1479
3,09/05/2023 14:40:24,Tom Donald,110477736284587831432,NaN,5,GoogleReviews\ChIJ-5kFm5vIxokR8VHXewCN8eY_Dela...,ChIJ-5kFm5vIxokR8VHXewCN8eY,Delancey Park,39.94408,-75.1479
4,08/18/2023 19:13:19,Evan Gray,117127716926195155753,NaN,5,GoogleReviews\ChIJ-5kFm5vIxokR8VHXewCN8eY_Dela...,ChIJ-5kFm5vIxokR8VHXewCN8eY,Delancey Park,39.94408,-75.1479


In [49]:
len(reviews_df['ParkName'].unique())

## Merged some park names to the larger parks but lat and long and place-id was original

164

In [50]:
reviews_df.to_csv('combined_reviews_with_park_info.csv', index=False)

## **Filter: parks with over 25 reviews**

In [55]:
reviews_df = reviews_df.drop('file_name', axis=1)

reviews_df = reviews_df.groupby('ParkName').filter(lambda x: x['review_text'].count() >= 25)


reviews_df.head()

,timestamp,author,user_id,review_text,rating,place_id,ParkName,Lat,Long
0,02/11/2024 15:59:38,Heidy Fernandez,103360372037270387417,NaN,5,ChIJ-5kFm5vIxokR8VHXewCN8eY,Delancey Park,39.94408,-75.1479
1,01/26/2024 22:16:50,R J,103202524746831207554,NaN,5,ChIJ-5kFm5vIxokR8VHXewCN8eY,Delancey Park,39.94408,-75.1479
2,12/30/2023 19:08:54,Eleni Belisonzi,106030427780813104531,A neighbor jewel.,5,ChIJ-5kFm5vIxokR8VHXewCN8eY,Delancey Park,39.94408,-75.1479
3,09/05/2023 14:40:24,Tom Donald,110477736284587831432,NaN,5,ChIJ-5kFm5vIxokR8VHXewCN8eY,Delancey Park,39.94408,-75.1479
4,08/18/2023 19:13:19,Evan Gray,117127716926195155753,NaN,5,ChIJ-5kFm5vIxokR8VHXewCN8eY,Delancey Park,39.94408,-75.1479


In [56]:
len(reviews_df['ParkName'].unique())

93

In [57]:
reviews_df.to_csv('combined_reviews_with_park_info_over_25reviews.csv', index=False)

# **Basic Exploration**

# **Sentiment**

## Apply Model

In [67]:
from transformers import pipeline

# Load the sentiment analysis pipeline
pipe = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")

# Make sure all entries in the 'text' column are strings
reviews_df['review_text'] = reviews_df['review_text'].astype(str)

# Apply the pipeline to each review
reviews_df['review_predictions'] = reviews_df['review_text'].apply(lambda x: pipe(x, truncation=True)) 

# Extract the label and score from the predictions
reviews_df['review_label'] = reviews_df['review_predictions'].apply(lambda x: x[0]['label'])
reviews_df['review_score'] = reviews_df['review_predictions'].apply(lambda x: x[0]['score'])

In [69]:
reviews_df.head()

,timestamp,author,user_id,review_text,rating,file_name,place_id,ParkName_x,Lat_x,Long_x,ParkName_y,Lat_y,Long_y,review_predictions,review_label,review_score
0,02/11/2024 15:59:38,Heidy Fernandez,103360372037270387417,nan,5,reviews\ChIJ-5kFm5vIxokR8VHXewCN8eY_Delancey_P...,ChIJ-5kFm5vIxokR8VHXewCN8eY,Delancey Park,39.94408,-75.1479,Delancey Park,39.94408,-75.1479,"[{'label': 'POSITIVE', 'score': 0.954146504402...",POSITIVE,0.954147
1,01/26/2024 22:16:50,R J,103202524746831207554,nan,5,reviews\ChIJ-5kFm5vIxokR8VHXewCN8eY_Delancey_P...,ChIJ-5kFm5vIxokR8VHXewCN8eY,Delancey Park,39.94408,-75.1479,Delancey Park,39.94408,-75.1479,"[{'label': 'POSITIVE', 'score': 0.954146504402...",POSITIVE,0.954147
2,12/30/2023 19:08:54,Eleni Belisonzi,106030427780813104531,A neighbor jewel.,5,reviews\ChIJ-5kFm5vIxokR8VHXewCN8eY_Delancey_P...,ChIJ-5kFm5vIxokR8VHXewCN8eY,Delancey Park,39.94408,-75.1479,Delancey Park,39.94408,-75.1479,"[{'label': 'POSITIVE', 'score': 0.995235741138...",POSITIVE,0.995236
3,09/05/2023 14:40:24,Tom Donald,110477736284587831432,nan,5,reviews\ChIJ-5kFm5vIxokR8VHXewCN8eY_Delancey_P...,ChIJ-5kFm5vIxokR8VHXewCN8eY,Delancey Park,39.94408,-75.1479,Delancey Park,39.94408,-75.1479,"[{'label': 'POSITIVE', 'score': 0.954146504402...",POSITIVE,0.954147
4,08/18/2023 19:13:19,Evan Gray,117127716926195155753,nan,5,reviews\ChIJ-5kFm5vIxokR8VHXewCN8eY_Delancey_P...,ChIJ-5kFm5vIxokR8VHXewCN8eY,Delancey Park,39.94408,-75.1479,Delancey Park,39.94408,-75.1479,"[{'label': 'POSITIVE', 'score': 0.954146504402...",POSITIVE,0.954147
